In [ ]:
%matplotlib notebook
import scipy.interpolate
import scipy.special
import scipy.constants
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas

import strawb.calibration

import plotly.express as px

import os

path_data = os.path.join(strawb.Config.repository_home, 'src/strawb/calibration')
save_dbs = False  # enable, if you want to store the data in the repro

# Water
## Define publication informations

In [ ]:
# Publications listed at: https://omlc.org/spectra/water/data
publication = [
    ['bricaud95', 'plankton', 
     """A. Bricaud and M. Babin and A. Morel and H. Claustre, 
     "Variability in the chlorophyll-specific absorption coefficients of natural 
     phytoplankton: analysis and parameterization,"
     Journal of Geophysical Research, 100, 13321-13332, (1995)."""],
    
    ['buiteveld94', 'pure water', 
     """H. Buiteveld and J. M. H. Hakvoort and M. Donze, 
     "The optical properties of pure water,"
     in SPIE Proceedings on Ocean Optics XII, edited by J. S. Jaffe, 2258, 174--183, (1994)."""],
    
    ['hale73', 'pure water', 
     """G. M. Hale and M. R. Querry, 
     "Optical constants of water in the 200nm to 200 micron wavelength region,"
     Appl. Opt., 12, 555--563, (1973)."""],
    
    ['irvine68', 'pure water', 
     """W. M. Irvine and J. B. Pollack, 
     "Infrared optical properties of water and ice spheres," Icarus, 8, 324--360, (1968)."""],

    ['kopelevich76', 'pure water', 
     """O. V. Kopelevich, 
     "Optical properties of pure water in the 250-600nmrange,"
     Opt. Spectrosc., 41, 391--392, (1976)."""],
    
    ['kou93b', 'pure water', 
     """L. Kou and D. Labrie and P. Chylek, 
     "Refractive indices of water and ice in the 0.65-2.5um spectral range,"
     Appl. Opt., 32, 3531--3540, (1993)."""],

    ['morel77', 'pure water', 
     """A. Morel and L. Prieur, 
     "Analysis of variations in ocean color,"
     Limnol. Oceanogr., 22, 709--722, (1977).
     Comment: explains colorshift of whater whith plankton and unorganic material"""],
    
    ['palmer74', '', 
     """K. F. Palmer and D. Williams, 
     "Optical properties of water in the near infrared,"
     J. Opt. Soc. Am., 64, 1107--1110, (1974)."""],


    ['pope93', 'pure water', 
     """R. M. Pope, 
     "Optical absorption of pure water and sea water using the integrating cavity absorption meter,"
     Texas A&M University, (1993)."""],

    ['pope97', 'pure water', 
     """R. M. Pope and E. S. Fry, 
     "Absorption spectrum (380-700nm) of pure water. II. Integrating cavity measurements,"
     Appl. Opt., 36, 8710--8723, 1997).
     Comment: Table with absorption Coefficients and Standard Deviations"""],
 
    ['querry78','pure water', 
     """Marvin R. Querry, Philip G. Cary, and Richard C. Waring,
     "Split-pulse laser method for measuring attenuation coefficients of transparent liquids: application to deionized filtered water in the visible region"
     """],
    
    ['querry91', 'pure water', 
     """M. R. Querry, D. M. Wieliczka, D. J. Segelstein, 
     "Water (H2O)," 
     Handbook of Optical Constants of Solids II, , 1059-1077 (1991)"""],

    ['segelstein81','pure water',     
     """D. J. Segelstein, 
     "The complex refractive index of water," University of
     Missouri-Kansas City, (1981)."""],
    
    # -> where to get?
    ['shifrin88','', 
     """K. S. Shifrin, 
     "Physical Optics of Ocean Water,"
     (1988)"""],

    ['smith81', 'pure water',
     """R. C. Smith and K. S. Baker, 
     "Optical properties of the clearest natural waters (200-800nm),"
     Appl. Opt., 20, 177--184, (1981)."""],

    ['sogandares97', 'pure water',
     """F. M. Sogandares and E. S. Fry, 
     "Absorption spectrum (340-640nm) of pure water. I. Photothermal Measurements,"\
     Appl. Opt., 36, 8699--8709, (1997)."""],

    # -> where to get?
    ['sullivan63', '',
     """S. A. Sullivan, 
     "Experimental study of the absorption in distilled water, artificial sea water, 
     and heavy water in the visible region of the spectrum,"
     Opt. Soc. Am. J., 53, 962--968, (1963)."""],

    ['tam79', '',
     """A. C. Tam and C. K. N. Patel, 
     "Optical absorption of light and heavy water by laser optoacoustic spectroscopy,"
     Appl. Opt., 18, 3348--3358, (1979)."""],

    ['warren95', 'ice',
     """S. G. Warren, 
     "Optical constants of ice from the ultraviolet to the microwave,"
     Appl. Opt., 23, 1026--1225, (1984).
     This data is based on the revised 1995 data of Warren.
     It was revised from the original data in the 1984
     paper to account for misinterpretations of data by others.
     It also includes new data from Kou."""],

    ['wieliczka89', '',
     """D. M. Wieliczka and S. Weng and M. R. Querry, 
     "Wedge shaped cell for highly absorbent liquids: infrared optical constants of water,"
     Appl. Opt., 28, 1714--1719, (1989)."""],

    ['zolotarev69', '', 
     """V. M. Zolotarev, B. A. Mikhilov, L. L. Alperovich, and S. I. Popov,
     "Dispersion and absorption of liquid water in the infrared and radio
     regions of the spectrum," Optics and Spectroscopy, 27, 430--432, (1969)."""],
    
    ['morel07', 'pure water and pure sea water',
    """André Morel, Bernard Gentili, Hervé Claustre, Marcel Babin, Annick Bricaud, Joséphine Ras, Fanny Tièche
    Optical properties of the "clearest" natural waters
    (2007)"""],
    
    ['quickenden80', 'pure water', 
     """Quickenden and Irvin,
     The column denoted aw2 corresponds to the absorption values deduced from the 
     attenuation values between 196 nm and 320 nm.
     (1980)."""],
    
    ['straw', 'sea water', """STRAW absorption values"""],
    
    # excluded as normalized and plankton
#     ['prieur81', 'plankton', 
#      """Prieur, L.  and Sathyendranath, S. 1981, An optical classification of coastal
#      and oceanic waters based on the specific spectral absorption curves of 
#      phytoplankton pigments dissolved organic matter, and other particulate
#      materials, Limnol. Oceanogr. 26(4), 671-689 for 400-700 nm
#      Normalized (to 440 nm) NONDIMENSIONAL chlorophyll-specific
#      absorption coefficient (extended by eye to 350 nm by Mobley)"""], 
]

## Store the publication information

In [ ]:
df_publication = pandas.DataFrame(publication,
                                  columns=['publication', 'medium', 'reference'])
df_publication.set_index('publication', inplace=True)

if save_dbs:
    df_publication.to_csv(os.path.join(path_data, 'water_publication.csv'))
                          
df_publication

## Get the data from https://omlc.org/spectra/water

In [ ]:
import ssl
import io
import urllib3
import tqdm.notebook
import requests

# to ignore the https warning
urllib3.disable_warnings()
# that pandas https works
ssl._create_default_https_context = ssl._create_unverified_context

def get_table(url):
    """Gets data of the url and parsed as a DataFrame."""
    r = requests.get(url, verify=False)
    i = 0
    for i, line_i in enumerate(r.iter_lines()):
        if line_i.decode(errors='ignore').replace(' ','').split('\t') == ['(nm)', '(1/cm)']:
#             print(i, line_i)
            break
        else:
            pass
#             print(line_i.decode(errors='ignore').split('\t'))

    text = r.text.replace('\t\n', '\n').replace(' ', '')
#     print(text.splitlines()[:i], text.splitlines()[i-1:i+1])
    df_i = pandas.read_table(io.StringIO('\n'.join(text.splitlines()[i-1:])), 
                             header=[0,1], 
                             delimiter='\t')
    
    # some have column lambda instead of wavelength, rename it
    df_i.rename(columns={"lambda": "wavelength"},inplace=True)
    df_i = df_i.droplevel(level=1, axis=1)
    df_i.absorption = df_i.absorption * 100  # 100: 1/cm -> 1/m
    return df_i

# load data for the publications
df_dict = {}
for pub_i in tqdm.notebook.tqdm(publication):
    # get values direct for some publications - exclude
    if pub_i[0] in ['morel77', 'morel07', 'straw', 'quickenden80']:
        continue
    df_i = get_table(f'https://omlc.org/spectra/water/data/{pub_i[0]}.txt')
    df_i['publication'] = pub_i[0]
    df_i['medium'] = pub_i[1]
    df_dict.update({pub_i[0]: df_i})

## Add other publications

In [ ]:
# 'morel77.txt'
# https://aslopubs.onlinelibrary.wiley.com/doi/pdf/10.4319/lo.1977.22.4.0709
# Attenuation (c_w), scattering (b_w), and absorption (a_w) coefficients 
# for optically (and chemically) pure wate
# [[Wavelength [nm], Attenuation (c_w) [m-1], scattering (b_w) [m-1], absorption (a_w) [m-1]]]
df_dict['morel77'] = pandas.DataFrame(
    columns = ['wavelength', 'attenuation', 'scattering', 'absorption'],
    data=[[380, 0.030, 0.0073, 0.023],
          [390, 0.027, 0.0066, 0.020],
          [400, 0.024, 0.0058, 0.018],
          [410, 0.022, 0.0052, 0.017],
          [420, 0.021, 0.0047, 0.016],
          [430, 0.013, 0.0043, 0.015],
          [440, 0.019, 0.0039, 0.015],
          [450, 0.018, 0.0035, 0.015],
          [460, 0.019, 0.0032, 0.016],
          [470, 0.019, 0.0029, 0.016],
          [480, 0.021, 0.0027, 0.018],
          [490, 0.022, 0.0024, 0.020],
          [500, 0.028, 0.0022, 0.026],
          [510, 0.038, 0.0020, 0.036],
          [520, 0.050, 0.0018, 0.048],
          [530, 0.053, 0.0017, 0.051],
          [540, 0.058, 0.0016, 0.056],
          [550, 0.066, 0.0015, 0.064],
          [560, 0.072, 0.0013, 0.071],
          [570, 0.081, 0.0013, 0.080],
          [580, 0.109, 0.0012, 0.108],
          [590, 0.158, 0.0011, 0.157],
          [600, 0.245, 0.00101, 0.245],
          [610, 0.290, 0.00094, 0.290],
          [620, 0.310, 0.00088, 0.310],
          [630, 0.320, 0.00088, 0.320],
          [640, 0.330, 0.00076, 0.330],
          [650, 0.350, 0.00071, 0.350],
          [660, 0.410, 0.00067, 0.410],
          [670, 0.450, 0.00063, 0.450],
          [680, 0.450, 0.00059, 0.450],
          [690, 0.500, 0.00055, 0.500],
          [700, 0.650, 0.00052, 0.650]])
df_dict['morel77']['publication'] = 'morel77'
df_dict['morel77']['medium'] = 'pure water'

In [ ]:
# Booth datasets come from:
# André Morel, Bernard Gentili, Hervé Claustre, Marcel Babin, Annick Bricaud, Joséphine Ras, Fanny Tièche
# Optical properties of the "clearest" natural waters
# (2007)

# Tabel 2: 
# Adopted values for the absorption coefficient (m21) between 300 nm and 500 nm for pure water (cf. Fig. 12), 
# as a function of the wavelength (l as nm). The column denoted aw1 corresponds to the absorption values proposed 
# by Buiteveld et al. (1994), which are based on the attenuation values determined by Boivin et al. (1986) 
# at 254, 313, and 366 nm. The values beyond 420 nm are those of Pope and Fry (1997).
# A smooth interpolation connects the value of Boivin at 366 nm and that of Pope and Fry at 420 nm.
# The column denoted aw2 corresponds to the absorption values deduced from the attenuation values obtained by
# Quickenden and Irvin (1980) (between 196 nm and 320 nm). The fitted values (between 300 and 320 nm; see Fig. 12) 
# are connected (at 366 nm) to the aw1 values. The scattering coefficients (at 20uC) for optically pure seawater, 
# bsw (m21) are derived from the values proposed by Buiteveld et al. (1994), increased by a factor of 1.30 to 
# account for the presence of salt (at a mean salinity of 36); the backscattering coefficient of pure seawater, bbsw (m21), is half the scattering coefficient. 
# The attenuation coefficient for downwelling irradiance in hypothetical pure seawater, Kdsw, displayed in Fig. 6, 
# is computed according to the approximation (underestimation) Kdsw1 5 aw1 + bbsw, or Kdsw2 5 aw2 + bbsw.
df_i = pandas.DataFrame(
    columns = ['wavelength', 'absorption', 'scattering'],
    data=[[300, 0.03820, 0.0226],
          [305, 0.03350, 0.0211],
          [310, 0.02880, 0.0197],
          [315, 0.02515, 0.0185],
          [320, 0.02150, 0.0173],
          [325, 0.01875, 0.0162],
          [330, 0.01600, 0.0152],
          [335, 0.01395, 0.0144],
          [340, 0.01190, 0.0135],
          [345, 0.01050, 0.0127],
          [350, 0.00910, 0.0121],
          [355, 0.00810, 0.0113],
          [360, 0.00710, 0.0107],
          [365, 0.00656, 0.0099],
          [370, 0.00602, 0.0095],
          [375, 0.00561, 0.0089],
          [380, 0.00520, 0.0085],
          [385, 0.00499, 0.0081],
          [390, 0.00478, 0.0077],
          [395, 0.00469, 0.0072],
          [400, 0.00460, 0.0069],
          [405, 0.00460, 0.0065],
          [410, 0.00460, 0.0062],
          [415, 0.00457, 0.0059],
          [420, 0.00454, 0.0056],
          [425, 0.00475, 0.0054],
          [430, 0.00495, 0.0051],
          [435, 0.00565, 0.0049],
          [440, 0.00635, 0.0047],
          [445, 0.00779, 0.0044],
          [450, 0.00922, 0.0043],
          [455, 0.00951, 0.0040],
          [460, 0.00979, 0.0039],
          [465, 0.01011, 0.0037],
          [470, 0.01060, 0.0035],
          [475, 0.01140, 0.0034],
          [480, 0.01270, 0.0033],
          [485, 0.01360, 0.0031],
          [490, 0.01500, 0.0030],
          [495, 0.01730, 0.0029],
          [500, 0.02040, 0.0027]])

# split it up for the two different mediums
df_i_2 = df_i.copy()
df_i_2['absorption'] = np.nan
df_i_2['medium'] = 'pure sea water'

df_i['scattering'] = np.nan
df_i['medium'] = 'pure water'
df_i = df_i.append(df_i_2)
df_i['publication'] = 'morel07'

df_dict['morel07'] = df_i

In [ ]:
# The column denoted aw2 corresponds to the absorption values deduced from the 
# attenuation values obtained by Quickenden and Irvin (1980) (between 196 nm and 320 nm).
df_dict['quickenden80'] = pandas.DataFrame(
    columns = ['wavelength', 'absorption'],
    data=[[300, 0.01150,],
          [305, 0.01100,],
          [310, 0.01050,],
          [315, 0.01013,],
          [320, 0.00975,],
          [325, 0.00926,],
          [330, 0.00877,],
          [335, 0.00836,],
          [340, 0.00794,],
          [345, 0.00753,],
          [350, 0.00712,],
          [355, 0.00684,],
          [360, 0.00656,],
          [365, 0.00629,],
          [370, 0.00602,]
         ])
df_dict['quickenden80']['publication'] = 'quickenden80'
df_dict['quickenden80']['medium'] = 'pure water'

In [ ]:
df_i = pandas.DataFrame(
    columns = ['wavelength', 'attenuation'],
    data=[[365, 1./10.4],  # 10.4+.4-.3 m
          [ 400, 1./14.6],  # 14.6+.4-.6 m
          [ 450, 1./27.7],  # 27.7+1.9-1.3 m
          [ 585, 1./7.1]]  # 7.1+.4-.3 m
    )

# of simplification add it as absorption also this is not 100% correct
df_i['absorption'] = df_i['attenuation'] 
df_i['publication'] = 'straw'
df_i['medium'] = 'sea water'
df_dict['straw'] = df_i

# straw_it = scipy.interpolate.interp1d(straw_wl,
#                                       BK7.absorption2transmittance(40., straw_abs),
#                                       fill_value="extrapolate") 
straw_it = scipy.interpolate.interp1d(df_dict['straw'].wavelength, 1./df_dict['straw'].attenuation,
                                      fill_value="extrapolate", kind='quadratic') # cubic

## Store the data

In [ ]:
# first combine the data to a single dataframe
df_data = pandas.concat(df_dict, ignore_index=True)

if save_dbs:
    df_data.to_csv(os.path.join(path_data, 'water_data.csv'))
    
# pandas.read_csv(os.path.join(path_data, 'water_data.csv'), index_col='Unnamed: 0')

# Plotting
Prepare the data
- caluclate absorption length
- normalized absorption length with the max. in the range 300-700nm
- wavelength with max absorption length in the range 300-700nm

In [ ]:
df_all = pandas.concat(df_dict, ignore_index=True)

# get absorption length
df_all['abs_len']  = 1./df_all['absorption']
df_all.loc[df_all.abs_len.isna(), 'abs_len']  = 1./df_all.loc[df_all.abs_len.isna(), 'attenuation']

mask = df_all.wavelength > 200
mask &= df_all.wavelength < 700
mask &= ~df_all.publication.isin(['bricaud95', 'kou93b'])

gb = df_all[mask].groupby('publication')

df_all['wavelength_max']= df_all.wavelength.loc[df_all.merge(gb.abs_len.idxmax(),
                                                             on='publication',
                                                             how='left')['abs_len_y'].fillna(0)].values

df_all['abs_len_normalized']= df_all.merge(gb.abs_len.max(), 
                                           on='publication', 
                                           how='left')['abs_len_y'].fillna(1)
df_all['abs_len_normalized'] = df_all['abs_len'] / df_all['abs_len_normalized']

## Plot all data

In [ ]:
px.line(df_all[df_all.wavelength<1e4], x='wavelength', y='abs_len', color='publication',
        hover_data=['medium'], log_y=True,)

## Plot the range 200-700nm

In [ ]:
px.line(df_all[mask], x='wavelength', y='abs_len', color='publication',
        hover_data=['medium'], log_y=True,)

## Normalized with the maximum in the range

In [ ]:
px.line(df_all[mask],
        x=df_all.wavelength[mask],
        y='abs_len_normalized', color='publication', hover_data=['medium'])

## Normalzied and relative to the maximum position, i.e. the maxium of all lines is at [0,1]

In [ ]:
px.line(df_all[mask],
        x=df_all.wavelength[mask]-df_all.wavelength_max[mask],
        y='abs_len_normalized', color='publication', hover_data=['medium'])

# Plankton !Data not added to the repro!


$a^*_{ph}(\lambda) = A(\lambda) {chl}^{-B(\lambda)}$
- $chl$: chlorophyll a concentration [$\frac{mg}{m^3}$]
- $a_{ph}$: absorption coefficient of phytoplanktion [$\frac{1}{m}$]
- $a^*_{ph}(\lambda)$: chl a-specific absorption coefficient of phytoplankton [$\frac{m^2}{mg}$]
    - $a^*_{ph}(\lambda) = \frac{a_{ph}(\lambda)}{chl}$
    - alter ver. from text: absorption cross section of algea per mass unit of chl [$ \frac{m^2\; mg}{{chl \; a}^{-1}}$] 
- $A(\lambda)$, $B(\lambda)$: wavelength dependent parameters
- $r^2$: determination coefficient on the log-transformed data

In [ ]:
# A. Bricaud and M. Babin and A. Morel and H. Claustre, "Variability in
# the chlorophyll-specific absorption coefficients of natural
# phytoplankton: analysis and parameterization," Journal of Geophysical
# Research, 100, 13321-13332, (1995).

# [nm],     A,     B, r**2 
df_chlorophyll = pandas.DataFrame(
    columns = ['wavelength', 'A', 'B', 'r**2'],
    data=[[400, 0.0263, 0.282, 0.702],
    [404, 0.0280, 0.282, 0.706],
    [408, 0.0301, 0.282, 0.710],
    [412, 0.0323, 0.286, 0.718],
    [416, 0.0342, 0.293, 0.725],
    [420, 0.0356, 0.299, 0.733],
    [424, 0.0362, 0.313, 0.746],
    [428, 0.0376, 0.317, 0.749],
    [432, 0.0391, 0.318, 0.750],
    [436, 0.0399, 0.328, 0.757],
    [440, 0.0403, 0.332, 0.762],
    [444, 0.0390, 0.348, 0.774],
    [448, 0.0375, 0.360, 0.783],
    [452, 0.0365, 0.362, 0.783],
    [456, 0.0354, 0.367, 0.789],
    [460, 0.0350, 0.365, 0.789],
    [464, 0.0343, 0.368, 0.792],
    [468, 0.0335, 0.369, 0.793],
    [472, 0.0325, 0.371, 0.792],
    [476, 0.0312, 0.378, 0.793],
    [480, 0.0301, 0.377, 0.791],
    [484, 0.0290, 0.376, 0.788],
    [488, 0.0279, 0.369, 0.783],
    [492, 0.0267, 0.356, 0.774],
    [496, 0.0249, 0.341, 0.763],
    [500, 0.0230, 0.321, 0.747],
    [504, 0.0209, 0.300, 0.722],
    [508, 0.0189, 0.275, 0.686],
    [512, 0.0171, 0.249, 0.641],
    [516, 0.0156, 0.224, 0.578],
    [520, 0.0143, 0.196, 0.498],
    [524, 0.0131, 0.173, 0.417],
    [528, 0.0121, 0.151, 0.332],
    [532, 0.0113, 0.129, 0.248],
    [536, 0.0104, 0.109, 0.176],
    [540, 0.0097, 0.090, 0.116],
    [544, 0.0090, 0.073, 0.074],
    [548, 0.0083, 0.059, 0.044],
    [552, 0.0076, 0.044, 0.023],
    [556, 0.0068, 0.027, 0.007],
    [560, 0.0062, 0.016, 0.002],
    [564, 0.0057, 0.010, 0.001],
    [568, 0.0054, 0.007, 0.000],
    [572, 0.0053, 0.011, 0.001],
    [576, 0.0052, 0.022, 0.004],
    [580, 0.0053, 0.035, 0.013],
    [584, 0.0055, 0.050, 0.028],
    [588, 0.0056, 0.065, 0.043],
    [592, 0.0057, 0.081, 0.072],
    [596, 0.0056, 0.093, 0.097],
    [600, 0.0054, 0.092, 0.086],
    [604, 0.0055, 0.086, 0.083],
    [608, 0.0056, 0.076, 0.067],
    [612, 0.0059, 0.069, 0.063],
    [616, 0.0062, 0.063, 0.056],
    [620, 0.0065, 0.064, 0.063],
    [624, 0.0067, 0.071, 0.083],
    [628, 0.0069, 0.076, 0.099],
    [632, 0.0073, 0.080, 0.109],
    [636, 0.0075, 0.088, 0.128],
    [640, 0.0077, 0.098, 0.149],
    [644, 0.0079, 0.113, 0.177],
    [648, 0.0081, 0.123, 0.195],
    [652, 0.0085, 0.125, 0.200],
    [656, 0.0095, 0.122, 0.206],
    [660, 0.0115, 0.121, 0.235],
    [664, 0.0144, 0.131, 0.308],
    [668, 0.0176, 0.143, 0.377],
    [672, 0.0197, 0.153, 0.424],
    [675, 0.0201, 0.158, 0.445],
    [402, 0.0271, 0.281, 0.702],
    [406, 0.0290, 0.281, 0.701],
    [410, 0.0313, 0.283, 0.713],
    [414, 0.0333, 0.291, 0.723],
    [418, 0.0349, 0.296, 0.729],
    [422, 0.0359, 0.306, 0.739],
    [426, 0.0369, 0.316, 0.747],
    [430, 0.0386, 0.314, 0.746],
    [434, 0.0395, 0.324, 0.754],
    [438, 0.0401, 0.332, 0.761],
    [442, 0.0398, 0.339, 0.767],
    [446, 0.0383, 0.355, 0.779],
    [450, 0.0371, 0.359, 0.781],
    [454, 0.0358, 0.366, 0.788],
    [458, 0.0351, 0.368, 0.791],
    [462, 0.0347, 0.366, 0.791],
    [466, 0.0339, 0.369, 0.793],
    [470, 0.0332, 0.368, 0.792],
    [474, 0.0318, 0.375, 0.793],
    [478, 0.0306, 0.379, 0.793],
    [482, 0.0296, 0.377, 0.790],
    [486, 0.0285, 0.373, 0.786],
    [490, 0.0274, 0.361, 0.779],
    [494, 0.0258, 0.349, 0.770],
    [498, 0.0240, 0.332, 0.756],
    [502, 0.0220, 0.311, 0.735],
    [506, 0.0199, 0.288, 0.706],
    [510, 0.0180, 0.260, 0.664],
    [514, 0.0163, 0.237, 0.615],
    [518, 0.0149, 0.211, 0.541],
    [522, 0.0137, 0.184, 0.459],
    [526, 0.0126, 0.162, 0.374],
    [530, 0.0117, 0.139, 0.287],
    [534, 0.0108, 0.119, 0.211],
    [538, 0.0100, 0.100, 0.147],
    [542, 0.0093, 0.081, 0.092],
    [546, 0.0086, 0.066, 0.057],
    [550, 0.0080, 0.052, 0.033],
    [554, 0.0072, 0.036, 0.014],
    [558, 0.0065, 0.016, 0.002],
    [562, 0.0059, 0.013, 0.001],
    [566, 0.0055, 0.007, 0.000],
    [570, 0.0053, 0.005, 0.000],
    [574, 0.0052, 0.018, 0.003],
    [578, 0.0052, 0.028, 0.007],
    [582, 0.0054, 0.040, 0.016],
    [586, 0.0055, 0.056, 0.033],
    [590, 0.0056, 0.073, 0.058],
    [594, 0.0056, 0.088, 0.084],
    [598, 0.0055, 0.095, 0.098],
    [602, 0.0054, 0.088, 0.078],
    [606, 0.0055, 0.082, 0.078],
    [610, 0.0057, 0.071, 0.060],
    [614, 0.0060, 0.066, 0.062],
    [618, 0.0063, 0.064, 0.061],
    [622, 0.0066, 0.068, 0.073],
    [626, 0.0068, 0.074, 0.092],
    [630, 0.0071, 0.078, 0.104],
    [634, 0.0074, 0.084, 0.119],
    [638, 0.0076, 0.093, 0.138],
    [642, 0.0078, 0.105, 0.164],
    [646, 0.0080, 0.119, 0.189],
    [650, 0.0083, 0.124, 0.197],
    [654, 0.0089, 0.124, 0.203],
    [658, 0.0104, 0.120, 0.218],
    [662, 0.0129, 0.125, 0.269],
    [666, 0.0161, 0.137, 0.345],
    [670, 0.0189, 0.149, 0.404],
    [674, 0.0201, 0.157, 0.439],
    [676, 0.0200, 0.159, 0.445],
    [678, 0.0193, 0.158, 0.444],
    [682, 0.0166, 0.148, 0.406],
    [686, 0.0124, 0.124, 0.315],
    [690, 0.0083, 0.086, 0.164],
    [694, 0.0054, 0.042, 0.036],
    [698, 0.0036,-0.016, 0.003],
    [680, 0.0182, 0.155, 0.433],
    [684, 0.0145, 0.138, 0.368],
    [688, 0.0102, 0.107, 0.247],
    [692, 0.0067, 0.065, 0.094],
    [696, 0.0044, 0.015, 0.004],
    [700, 0.0030,-0.034, 0.012],
    ])
df_chlorophyll.sort_values('wavelength', inplace=True)

def a(wavelength, A, B, chl):
    return A * chl**(-B+1)

plt.figure()

for i in [0.03, 0.1, 0.3, 1, 3, 10]:
    plt.plot(df_chlorophyll.wavelength, 
             a(df_chlorophyll.wavelength, df_chlorophyll.A, df_chlorophyll.B, i), 
             label=i)
    
plt.grid()
plt.legend()